In [ ]:
!pip install pandas

In [252]:
# Common imports
import numpy as np
import os
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [ ]:
DATASET_PATH = "./datasets"

def load_covid_data(filename, dataset_path=DATASET_PATH):
    xlsx_path = os.path.join(dataset_path, filename)
    return pd.read_excel(xlsx_path)

In [286]:
def convert_age(age):
  if type(age) == str or age == None or math.isnan(age):
    return 1
  else:
    return int(age)

def parse_travel_history(th):
  if type(th) == str and th.startswith('n'):
    return True
  return False


def parse_public_transport(th):
  if type(th) == str:
    th = th.lower()
    if "masina" in th or th.startswith("n"):
      return False
    return True
  return False

def parse_result(th):
  if type(th) == str:
    th = th.lower()
    if th.startswith("n"):
      return False
    return True
  return False

def parse_contact(th):
  if type(th) == str:
    th = th.lower()
    if "tie" in th:
      return 2
    elif "nu" in th or "neag" in th:
      return 0
    return 1
  return 0


def parse_symptoms(th):

  if type(th) is not str:
    return ''

  rv = dict() 
  th = th.lower()
  
  rv['febra'] = 'febr' in th or 'temp' in th
  rv['tuse'] = 'tuse' in th
  rv['dispnee'] = 'dispnee' in th
  rv['fatigabilitate'] = 'fatigabilitate' in th or 'astenie' in th
  rv['diaree'] = 'diaree' in th
  rv['dureri'] = 'dur' in th
  rv['simturi'] = 'gust' in th or 'miros' in th or 'simt' in th
  rv['greturi'] = 'gre' in th or 'vars' in th

  return rv

def parse_suspect(th):

  if type(th) is not str:
    return 2

  if 'sus' in th or 'cov' in th:
    return 1

  return 0

full_data = load_covid_data("mps.dataset.xlsx")
full_data.rename(columns={'vârstă': 'age'}, inplace=True)
full_data.rename(columns={'istoric de călătorie': 'travel_history'}, inplace=True)
full_data.rename(columns={'mijloace de transport folosite': 'public_transport'}, inplace=True)
full_data.rename(columns={'confirmare contact cu o persoană infectată': 'contact'}, inplace=True)
full_data.rename(columns={'rezultat testare': 'result'}, inplace=True)
full_data.rename(columns={'simptome raportate la internare': 'symptoms'}, inplace=True)
full_data.rename(columns={'diagnostic și semne de internare': 'suspect'}, inplace=True)


full_data.replace(['F','FEMININ     ','FEMININ'],'FEMININ', inplace=True)
full_data.replace(['masculin'],'MASCULIN', inplace=True)
full_data = full_data[full_data['sex'].notna()]
full_data['age'] = full_data['age'].map(convert_age)
full_data = full_data[full_data['age'] < 100]
full_data = full_data[full_data['result'] != 'NECONCLUDENT']
full_data['travel_history'] = full_data['travel_history'].map(parse_travel_history)
full_data['public_transport'] = full_data['public_transport'].map(parse_public_transport)
full_data['result'] = full_data['result'].map(parse_result)
full_data['contact'] = full_data['contact'].map(parse_contact)
full_data['simptome declarate'] = full_data['simptome declarate'].map(str)
full_data['symptoms'] = full_data['symptoms'].astype(str) + ' ' +full_data['simptome declarate']
full_data['symptoms'] = full_data['symptoms'].map(parse_symptoms)
full_data['suspect'] = full_data['suspect'].map(parse_suspect)

full_data = pd.concat([full_data, full_data['symptoms'].apply(pd.Series)], axis = 1).drop('symptoms', axis = 1)


_ = full_data.pop('instituția sursă')
_ = full_data.pop('dată debut simptome declarate')
_ = full_data.pop('dată internare')
_ = full_data.pop('data rezultat testare')
_ = full_data.pop('simptome declarate')


In [234]:
full_data.describe(include='all')

,sex,age,suspect,travel_history,public_transport,contact,result,febra,tuse,dispnee,fatigabilitate,diaree,dureri,simturi,greturi
count,6330,6330.000000,6330,6330,6330,6330.000000,6330,6330,6330,6330,6330,6330,6330,6330,6330
unique,2,NaN,2,2,2,NaN,2,2,2,2,2,2,2,2,2
top,FEMININ,NaN,False,False,False,NaN,False,False,False,False,False,False,False,False,False
freq,3657,NaN,6275,6178,6311,NaN,5708,6027,6081,6078,6229,6321,6151,6321,6247
mean,NaN,46.484834,NaN,NaN,NaN,0.055924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,21.538961,NaN,NaN,NaN,0.315052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,32.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,47.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,61.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [271]:
full_data.pop('sex')
full_data.pop('age')
X = full_data
y = full_data.pop('result')

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)

rf = RandomForestClassifier(n_estimators=100, random_state=123456)
rf.fit(X_train, y_train)

KeyError: ignored

In [269]:
predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)
print(f'Mean accuracy score: {accuracy:.3}')

Mean accuracy score: 0.909


In [282]:
np.unique(predicted, return_counts=True)

(array([False,  True]), array([1572,   11]))